### Data extraction

In [1]:
from utils import fetch_log_returns
training_dataset = fetch_log_returns(start = '1995-01-01', end ='1995-12-31')

[*********************100%***********************]  5 of 5 completed


In [2]:
training_dataset.values

array([[-8.03420318e-04, -1.03968302e-02, -6.34465166e-03,
        -3.45697194e-03, -1.95645036e-02],
       [ 7.38303962e-04,  3.28007496e-03,  1.07261458e-02,
        -4.93917085e-03,  0.00000000e+00],
       [ 3.25539389e-04, -2.33000766e-03, -3.00612927e-03,
        -3.82611528e-03, -1.43458320e-02],
       ...,
       [ 3.74407369e-04, -1.62287613e-04,  4.93541396e-03,
         5.36316418e-03, -2.68870072e-02],
       [-6.67454827e-04, -2.87200507e-03,  8.16115201e-05,
        -6.95147773e-03,  1.73196363e-02],
       [ 2.94296798e-03, -5.82632554e-03,  3.42115394e-03,
        -2.50644600e-04,  8.74144700e-03]])

### Define the problem instance parameters

In [22]:
K = 1 # number of qubits assigned per asset 

NUM_ASSETS = len(training_dataset.columns) # number of assets
N = NUM_ASSETS*K # number of total qubits
NLAYERS = 2 # number of layers of the ansatz
NSHOTS = 10 # when measuring the ansatz

LAMBDA_1 = 1.0 # return penalty coefficient
LAMBDA_2 = 1.0 # risk penalty coefficient
SIGMA_TARGET = 0.1 # target volatility

from qibo.gates import U1, U2, U3, CZ, CNOT

TWO_QUBIT_GATES = {"CZ": CZ, "CNOT": CNOT} # tytpes of two-qubit gates

### Create the Ansatz

In [85]:
from qibo import models, gates

def compute_number_of_params(num_qubits, num_layers):
    return [0] * num_qubits * (2 * num_layers + 2 + num_layers)

def build_hardware_efficient_ansatz(num_qubits: int, params: [float] = [], num_layers: int = 1, two_gate: str = "CNOT"):
    c = models.Circuit(num_qubits)
    c.add([gates.U2(qubit, 0, 0) for qubit in range(num_qubits)])
    params = compute_number_of_params(num_qubits, num_layers)
    for _ in range(num_layers):
        c.add([gates.U2(qubit, 0, 0) for qubit in range(num_qubits)])
        c.add([TWO_QUBIT_GATES[two_gate](qubit, qubit+1)] for qubit in range(0,num_qubits-1))
        c.add([gates.U1(qubit,0) for qubit in range(num_qubits)])
            
    c.set_parameters(params)
    c.add([gates.M(qubit) for qubit in range(num_qubits)])
    return c

# Example usage
print(build_hardware_efficient_ansatz(num_qubits=5, num_layers=3).draw())


q0: ─U2─U2─o───────U1─U2─o───────U1─U2─o───────U1─M─
q1: ─U2─U2─X─o─────U1─U2─X─o─────U1─U2─X─o─────U1─M─
q2: ─U2─U2───X─o───U1─U2───X─o───U1─U2───X─o───U1─M─
q3: ─U2─U2─────X─o─U1─U2─────X─o─U1─U2─────X─o─U1─M─
q4: ─U2─U2───────X─U1─U2───────X─U1─U2───────X─U1─M─


### Building the Portfolio Optimization Hamiltonian

In [5]:

# Define the vector that contains the pauli opeations
import pandas as pd
def A(i: int, bit_string: list):
    return sum(2 ** (k - 2) * ((1 - bit_string[k + i*K]) / 2) for k in range(K))

# Return term

def return_cost_function(dataset: pd.DataFrame, bit_string: list[int]):
    h1 = 0
    for i, asset in enumerate(dataset.columns):
        h1 += A(i,bit_string) * dataset[asset].values
    return (-1)*sum(h1)


# Volatility term

def tilde_sigma(i: int,j: int, dataset: pd.DataFrame):
    if i==j: 
        return dataset.cov().values[i][j]
    elif i<j:
        return 2 * dataset.cov().values[i][j]
    else: 
        return 0

def risk_cost_function(dataset: pd.DataFrame, bit_string: list):
    h2 = 0
    for i in range(NUM_ASSETS):
        for j in range(NUM_ASSETS):  
            h2 += tilde_sigma(i,j,dataset) * A(i, bit_string) * A(i, bit_string) - SIGMA_TARGET ** 2

    return h2 ** 2

# # Third term: lambda_3 * (sum(A_i) - 1)^2
# term3 = lambda_3 * (sum(A[i] for i in range(N)) - 1) ** 2



In [6]:
import numpy as np
from qibo.optimizers import optimize
from utils import string_to_int_list
import qibo

def compute_return_energy(result: qibo.result.CircuitResult, dataset: pd.DataFrame): 
    return_energy = 0
    for bit_string, stat_freq in result.frequencies().items():
        return_energy += (stat_freq / NSHOTS) * return_cost_function(dataset,string_to_int_list(bit_string))
    return return_energy

def compute_risk_energy(result: qibo.result.CircuitResult, dataset: pd.DataFrame): 
    risk_energy = 0
    for bit_string, stat_freq in result.frequencies().items():
        risk_energy += (stat_freq / NSHOTS) * risk_cost_function(dataset,string_to_int_list(bit_string))
    return risk_energy
    
def compute_total_energy(parameters, dataset, nshots = NSHOTS):
    
    circuit = build_ansatz_circuit(parameters)
    
    # Measure the qubits quantum state
    result = circuit(nshots=nshots) 
    total_energy = LAMBDA_1 * compute_return_energy(result,dataset) + LAMBDA_2 * compute_risk_energy(result,dataset)
    
    return total_energy



# Optimize starting from a random guess for the variational parameters
initial_params = np.random.uniform(0, 2*np.pi, 2*N*NLAYERS + N)
# data = np.random.normal(0, 1, size=2**nqubits)

# perform optimization
best, params, extra = optimize(compute_total_energy, initial_params, args=(training_dataset))

# set final solution to circuit instance
print('Ground state:', best)

print('Optimal parameters', params)

print('Optimization process info', extra)

[Qibo 0.2.8|INFO|2024-10-02 16:51:30]: Using numpy backend on /CPU:0


Ground state: -0.015186763478192852
Optimal parameters [-2.76681648  6.30681763  6.37835661  6.30276102  6.65511022  1.62500973
  2.20707899  0.97029306  0.65138009  2.07480658  4.34147944  1.16653827
  2.59099218  6.68442965  6.90874322  2.46571327 10.75758374  5.20507667
  6.40296417  2.86535356  3.46175683 -1.41040839 -0.7845615   6.22317745
  6.9475845 ]
Optimization process info  message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -0.015186763478192852
       x: [-2.767e+00  6.307e+00 ...  6.223e+00  6.948e+00]
     nit: 2
   direc: [[ 1.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
           [ 0.000e+00  1.000e+00 ...  0.000e+00  0.000e+00]
           ...
           [ 0.000e+00  0.000e+00 ...  1.000e+00  0.000e+00]
           [ 0.000e+00  0.000e+00 ...  0.000e+00  1.000e+00]]
    nfev: 725


In [7]:
c = build_ansatz_circuit(initial_params)
result = c(nshots=10)
result

In [8]:
compute_energy_per_term(result, return_cost_function, training_dataset, string_to_int_list('10000'))

NameError: name 'compute_energy_per_term' is not defined